In [1]:
import torch
import numpy as np
import os
import torchvision
import crypten
import sklearn
import matplotlib.pyplot as plt
from torchvision import transforms
import tqdm
import crypten.mpc as mpc
import crypten.communicator as comm

/future/u/alexder/anaconda3/envs/cs356v2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
crypten.init()

ALICE = 0
BOB = 1

In [3]:
!ls

all_data.pth
all_targets.pth
all_val_data.pth
all_val_targets.pth
cifar10_raw_pytorch.ipynb
mnist_data_playground.ipynb
mnist_data_poisoned_playground.ipynb
mnist_train_playground_encrypted_inference.ipynb
mnist_train_playground.ipynb
mnist_train_poisoned_playground-Copy1.ipynb
mnist_train_poisoned_playground.ipynb
one_hot_target
poisoned_data_samples
scalar_target
tutorials


In [4]:
def train(model, all_data, all_targets, all_val_data, all_val_targets):
    
    model.train()
    
    criterion = torch.nn.CrossEntropyLoss()
    lr = 0.01
    batch_size = 16
    optimizer = torch.optim.SGD(model.parameters(), lr = lr, momentum=0.9)

    # Define training parameters
    num_epochs = 2
    val_every_n = 10
    num_batches = all_data.size(0) // batch_size
    
    batch_loss = []
    batch_acc = []
    
    for i in range(num_epochs): 
        
        for batch in range(num_batches):
            model.train()
            # define the start and end of the training mini-batch
            start, end = batch * batch_size, (batch + 1) * batch_size
                                    
            # construct CrypTensors out of training examples / labels
            x_train = all_data[start:end]
            y_train = torch.Tensor(all_targets[start:end]).long()
            optimizer.zero_grad()

            outputs = model(x_train)
            loss = criterion(outputs, y_train)
            loss.backward()
            batch_loss.append(loss)
            optimizer.step()
            
            if (batch % val_every_n == 0):
                print("Validating")
                acc = val(model, all_val_data, all_val_targets)
                batch_acc.append(acc)
                
    return batch_loss, batch_acc

In [5]:
def val(model, all_data, all_targets):
    
    model.eval()
    
    # Define training parameters
    batch_size = 8
    num_batches = all_data.size(0) // batch_size
    
    total_acc = 0
    for batch in range(num_batches):
        start, end = batch * batch_size, (batch + 1) * batch_size

        # construct CrypTensors out of training examples / labels
        x_val = all_data[start:end]
        y_val = torch.Tensor(all_targets[start:end]).long()

        outputs = model(x_val)
        acc = torch.sum(torch.argmax(outputs, dim = 1) == y_val)/ x_val.shape[0]
        total_acc += acc
        
    return total_acc / (num_batches)

In [6]:
!ls one_hot_target

data_samples_sub1000x  data_samples_sub50x


In [26]:
subsample = 0.02
            
@mpc.run_multiprocess(world_size=2)
def data_labeling_example():
    """Apply data labeling access control model"""
    # Alice loads features, Bob loads labels
    data_root_dir = './one_hot_target/data_samples_sub50x'
    data_alice_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/data_0.pth'), src=ALICE)
    targets_alice_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/targets_0.pth'), src=ALICE)
    
    data_bob_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/data_1.pth'), src=BOB)
    targets_bob_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/targets_1.pth'), src=BOB)
    
    val_data_alice_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/data_0.pth'), src=ALICE)
    val_targets_alice_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/targets_0.pth'), src=ALICE)
    
    val_data_bob_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/data_1.pth'), src=BOB)
    val_targets_bob_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/targets_1.pth'), src=BOB)
    
    model = torchvision.models.resnet50(num_classes = 10)
    model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    all_data = crypten.cat([data_alice_enc, data_bob_enc], dim=0)
    all_targets = crypten.cat([targets_alice_enc, targets_bob_enc], dim=0)
    
    all_val_data = crypten.cat([val_data_alice_enc, val_data_bob_enc], dim=0)
    all_val_targets = crypten.cat([val_targets_alice_enc, val_targets_bob_enc], dim=0)
    
    
    dummy_input = torch.empty(8, 1, 28, 28)
    resnet_plaintext = torchvision.models.resnet18(num_classes = 10)
    resnet_plaintext.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model = crypten.nn.from_pytorch(resnet_plaintext, dummy_input)
    model.encrypt()
    
    model.train()
    
    loss = crypten.nn.MSELoss()
    
    lr = 0.001
    num_epochs = 2
    batch_size = 8
    
    num_batches = all_data.size(0) // batch_size
    
    batch_losses = []
    
    for i in range(num_epochs):
        crypten.print(f"Epoch {i}")
        for batch in range(num_batches):
            
            start, end = batch * batch_size, (batch + 1) * batch_size
                                    
            # construct CrypTensors out of training examples / labels
            x_train = all_data[start:end]
            y_train = all_targets[start:end]
            #y_train = crypten.cryptensor(y_batch, requires_grad=True)
            
            # perform forward pass:
            output = model(x_train)
            loss_value = loss(output, y_train)
            
            # set gradients to "zero" 
            model.zero_grad()

            # perform backward pass: 
            loss_value.backward()

            # update parameters
            model.update_parameters(lr)
            
            # Print progress every batch:
            batch_loss = loss_value.get_plain_text().detach()
            batch_losses.append(batch_loss)
            crypten.print(f"\tBatch {(batch + 1)} of {num_batches} Loss {batch_loss.item():.4f}")
            
    np.save('batch_losses.npy', batch_losses)


In [27]:
data_labeling_example()

/future/u/alexder/anaconda3/envs/cs356v2/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/future/u/alexder/anaconda3/envs/cs356v2/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/future/u/alexder/anaconda3/envs/cs356v2/lib/python3.9

Epoch 0


/future/u/alexder/anaconda3/envs/cs356v2/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/future/u/alexder/anaconda3/envs/cs356v2/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or

	Batch 1 of 150 Loss 0.2669
	Batch 2 of 150 Loss 0.1692
	Batch 3 of 150 Loss 0.1690
	Batch 4 of 150 Loss 0.1908
	Batch 5 of 150 Loss 0.1483
	Batch 6 of 150 Loss 0.1294
	Batch 7 of 150 Loss 0.1069
	Batch 8 of 150 Loss 0.1088
	Batch 9 of 150 Loss 0.1519
	Batch 10 of 150 Loss 0.1020
	Batch 11 of 150 Loss 0.1399
	Batch 12 of 150 Loss 0.1118
	Batch 13 of 150 Loss 0.1091
	Batch 14 of 150 Loss 0.1335
	Batch 15 of 150 Loss 0.1222
	Batch 16 of 150 Loss 0.1056
	Batch 17 of 150 Loss 0.0927
	Batch 18 of 150 Loss 0.1151
	Batch 19 of 150 Loss 0.1095
	Batch 20 of 150 Loss 0.1138
	Batch 21 of 150 Loss 0.1182
	Batch 22 of 150 Loss 0.1271
	Batch 23 of 150 Loss 0.1127
	Batch 24 of 150 Loss 0.1035
	Batch 25 of 150 Loss 0.1063
	Batch 26 of 150 Loss 0.0871
	Batch 27 of 150 Loss 0.1095
	Batch 28 of 150 Loss 0.1255
	Batch 29 of 150 Loss 0.1177
	Batch 30 of 150 Loss 0.0969
	Batch 31 of 150 Loss 0.1044
	Batch 32 of 150 Loss 0.1020
	Batch 33 of 150 Loss 0.1076
	Batch 34 of 150 Loss 0.0985
	Batch 35 of 150 Loss 0

	Batch 131 of 150 Loss 0.1075
	Batch 132 of 150 Loss 0.0925
	Batch 133 of 150 Loss 0.1004
	Batch 134 of 150 Loss 0.1054
	Batch 135 of 150 Loss 0.1006
	Batch 136 of 150 Loss 0.1039
	Batch 137 of 150 Loss 0.0949
	Batch 138 of 150 Loss 0.1029
	Batch 139 of 150 Loss 0.0951
	Batch 140 of 150 Loss 0.1083
	Batch 141 of 150 Loss 0.0904
	Batch 142 of 150 Loss 0.1011
	Batch 143 of 150 Loss 0.0946
	Batch 144 of 150 Loss 0.0929
	Batch 145 of 150 Loss 0.0986
	Batch 146 of 150 Loss 0.0884
	Batch 147 of 150 Loss 0.0852
	Batch 148 of 150 Loss 0.1026
	Batch 149 of 150 Loss 0.1071
	Batch 150 of 150 Loss 0.1051


[None, None]

In [ ]:
all_data = torch.load('all_data.pth')
all_targets = torch.load('all_targets.pth')

all_val_data = torch.load('all_val_data.pth')
all_val_targets = torch.load('all_val_targets.pth')

In [ ]:


plt.imshow(all_data[19][0]), all_targets[19]
batch_losses, batch_accs = train(model, all_data, all_targets, all_val_data, all_val_targets)

In [ ]:
len(all_data)/8

In [ ]:
batch_losses = [loss.detach().numpy() for loss in batch_losses]
plt.plot(batch_losses)
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.title("Batch vs. Loss (without Poisoned Data)")

In [ ]:
batch_accs = [acc.detach().numpy() for acc in batch_accs]
plt.plot(batch_accs)
plt.xlabel("Batch")
plt.ylabel("Acc")
plt.title("Batch vs. Val Accuracy (Without Poisoned Data)")

In [ ]:
acc = val(model, all_val_data, all_val_targets)

In [ ]:
batch_accs